In [ ]:
import numpy as np
from time import sleep
import random

class ConnectSixGame:
    def __init__(self):
        """
        Initialize the Connect Six game with 8 rows and 9 columns.
        The board is represented as a numpy array with:
        0 = empty cell
        1 = player 1's disc
        2 = player 2's disc (or AI)
        """
        self.rows = 8
        self.cols = 9
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.game_over = False
        self.winner = None
        # Number of consecutive discs required to win
        self.connect_n = 6
        # Maximum depth for minimax algorithm
        self.max_depth = 3  # Reduced for better performance
        # Hint limits for Human vs Human mode
        self.player1_hints = 3
        self.player2_hints = 3

    def reset_game(self):
        """Reset the game board and game state"""
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.game_over = False
        self.winner = None
        self.player1_hints = 3
        self.player2_hints = 3

    def get_valid_moves(self):
        """
        Return a list of valid moves (rows with at least one empty cell)
        """
        return [row for row in range(self.rows) if self.is_valid_move(row)]

    def place_disc(self, row, player):
        """
        Place a disc in the leftmost empty position of the specified row
        Returns True if successful, False if the row is full
        """
        for col in range(self.cols):
            if self.board[row][col] == 0:
                self.board[row][col] = player
                # Check if this move caused a win
                if self.check_win(player):
                    self.game_over = True
                    self.winner = player
                elif self.is_board_full():
                    self.game_over = True
                    self.winner = 0
                return True
        return False

    def is_valid_move(self, row):
        """Check if a move is valid (row has at least one empty cell)"""
        if row < 0 or row >= self.rows:
            return False
        return 0 in self.board[row]

    def is_board_full(self):
        """Check if the board is completely full"""
        return not any(0 in row for row in self.board)

    def check_win(self, player):
        """
        Check if the specified player has won the game by having
        connect_n consecutive discs in any direction
        """
        # Check horizontal wins
        for r in range(self.rows):
            for c in range(self.cols - self.connect_n + 1):
                if all(self.board[r][c+i] == player for i in range(self.connect_n)):
                    return True

        # Check vertical wins
        for r in range(self.rows - self.connect_n + 1):
            for c in range(self.cols):
                if all(self.board[r+i][c] == player for i in range(self.connect_n)):
                    return True

        # Check diagonal (top-left to bottom-right)
        for r in range(self.rows - self.connect_n + 1):
            for c in range(self.cols - self.connect_n + 1):
                if all(self.board[r+i][c+i] == player for i in range(self.connect_n)):
                    return True

        # Check diagonal (bottom-left to top-right)
        for r in range(self.connect_n - 1, self.rows):
            for c in range(self.cols - self.connect_n + 1):
                if all(self.board[r-i][c+i] == player for i in range(self.connect_n)):
                    return True

        return False

    def evaluate_window(self, window, player):
        """
        Evaluate a window of connect_n cells for the heuristic function
        """
        opponent = 1 if player == 2 else 2
        player_count = np.count_nonzero(window == player)
        empty_count = np.count_nonzero(window == 0)
        opponent_count = np.count_nonzero(window == opponent)

        if player_count == self.connect_n:
            return 1000
        elif player_count == self.connect_n - 1 and empty_count == 1:
            return 100
        elif player_count == self.connect_n - 2 and empty_count == 2:
            return 10
        elif opponent_count == self.connect_n - 1 and empty_count == 1:
            return -200  # More urgent to block opponent
        elif opponent_count == self.connect_n - 2 and empty_count == 2:
            return -20
        return 0

    def score_position(self, player):
        """
        Calculate a score for the current board position from player's perspective
        """
        score = 0
        opponent = 1 if player == 2 else 2

        # Prefer center column
        center_col = self.cols // 2
        center_array = self.board[:, center_col]
        score += np.count_nonzero(center_array == player) * 3
        score -= np.count_nonzero(center_array == opponent) * 3

        # Score all possible windows
        directions = [
            (0, 1),   # Horizontal
            (1, 0),   # Vertical
            (1, 1),    # Diagonal /
            (1, -1)    # Diagonal \
        ]

        for dr, dc in directions:
            for r in range(self.rows):
                for c in range(self.cols):
                    if (0 <= r + (self.connect_n-1)*dr < self.rows and
                        0 <= c + (self.connect_n-1)*dc < self.cols):
                        window = [self.board[r + i*dr][c + i*dc] for i in range(self.connect_n)]
                        score += self.evaluate_window(window, player)

        return score

    def minimax(self, depth, alpha, beta, maximizing_player):
        """
        Minimax algorithm with Alpha-Beta pruning
        """
        valid_moves = self.get_valid_moves()
        current_player = 1 if maximizing_player else 2

        # Terminal node checks
        if self.check_win(1):
            return (None, 1000000)
        elif self.check_win(2):
            return (None, -1000000)
        elif self.is_board_full() or depth == 0:
            return (None, self.score_position(1))  # Score from AI's perspective

        if maximizing_player:
            value = float('-inf')
            best_move = random.choice(valid_moves) if valid_moves else None

            for move in valid_moves:
                # Make move
                board_copy = self.board.copy()
                self.place_disc(move, 1)

                # Recursive call
                new_score = self.minimax(depth-1, alpha, beta, False)[1]

                # Undo move
                self.board = board_copy
                self.game_over = False
                self.winner = None

                if new_score > value:
                    value = new_score
                    best_move = move

                alpha = max(alpha, value)
                if alpha >= beta:
                    break

            return best_move, value

        else:  # Minimizing player
            value = float('inf')
            best_move = random.choice(valid_moves) if valid_moves else None

            for move in valid_moves:
                # Make move
                board_copy = self.board.copy()
                self.place_disc(move, 2)

                # Recursive call
                new_score = self.minimax(depth-1, alpha, beta, True)[1]

                # Undo move
                self.board = board_copy
                self.game_over = False
                self.winner = None

                if new_score < value:
                    value = new_score
                    best_move = move

                beta = min(beta, value)
                if alpha >= beta:
                    break

            return best_move, value

    def get_ai_move(self, player):
        """Get the best move for the AI using minimax"""
        maximizing = (player == 1)
        move, _ = self.minimax(self.max_depth, float('-inf'), float('inf'), maximizing)
        return move if move is not None else random.choice(self.get_valid_moves())

    def display_board(self):
        """Display the current state of the board"""
        off = '\x1b[0m'
        red = '\x1b[31m'
        yel = '\x1b[33m'
        piece = '●'
        cells = ['·', red + piece + off, yel + piece + off]

        print('  ' + ' '.join(str(i+1) for i in range(self.cols)))
        for r in range(self.rows):
            print(f"{r+1} " + ' '.join([cells[self.board[r][c]] for c in range(self.cols)]))
        print('-' * (self.cols * 2 + 1))

    def play_human_vs_human(self):
        """Play a game of Connect Six: Human vs Human with optional AI hints (max 3 per player)"""
        print("\nWelcome to Connect Six: Human vs Human!")
        print("Connect six discs horizontally, vertically, or diagonally to win!")
        print("Player 1 is Red, Player 2 is Yellow")
        print("You will be inserting discs from left to right in rows.")
        print("Enter 'h' to get an AI hint for your move (max 3 hints per player).\n")

        current_player = 1  # Player 1 starts
        self.display_board()

        while not self.game_over:
            player_name = "Player 1 (Red)" if current_player == 1 else "Player 2 (Yellow)"
            hints_remaining = self.player1_hints if current_player == 1 else self.player2_hints
            move = -1
            while not self.is_valid_move(move):
                try:
                    user_input = input(f"\n{player_name}'s turn! Choose a row (1-8) or 'h' for hint ({hints_remaining} left): ").strip().lower()

                    if user_input == 'h':
                        # Check if player has hints remaining
                        if hints_remaining > 0:
                            print("AI is calculating a hint...")
                            sleep(0.5)
                            hint_move = self.get_ai_move(current_player)
                            if hint_move is not None:
                                print(f"AI suggests placing a disc in row {hint_move + 1}")
                                # Decrement hint count
                                if current_player == 1:
                                    self.player1_hints -= 1
                                else:
                                    self.player2_hints -= 1
                            else:
                                print("No valid moves available for a hint.")
                        else:
                            print("You've used all your hints!")
                        continue  # Prompt for input again

                    move = int(user_input) - 1
                    if not self.is_valid_move(move):
                        print("Invalid move. That row is full or doesn't exist.")
                except ValueError:
                    print("Please enter a number between 1 and 8 or 'h' for a hint.")

            # Place the disc
            self.place_disc(move, current_player)
            self.display_board()

            if self.game_over:
                break

            # Switch players
            current_player = 3 - current_player  # Switches between 1 and 2

        # Game over message
        if self.winner == 1:
            print("\nPlayer 1 (Red) wins!")
        elif self.winner == 2:
            print("\nPlayer 2 (Yellow) wins!")
        else:
            print("\nIt's a draw!")

    def play_human_vs_ai(self):
        """Play a game of Connect Six: Human vs AI"""
        print("\nWelcome to Connect Six!")
        print("Connect six discs horizontally, vertically, or diagonally to win!")
        print("You are Player 2 (Yellow) and the AI is Player 1 (Red)")
        print("You will be inserting discs from left to right in rows.\n")

        first_player = input("Would you like to go first? (y/n): ").lower()
        human_turn = first_player == 'y'

        self.display_board()

        while not self.game_over:
            if human_turn:
                # Human's turn
                move = -1
                while not self.is_valid_move(move):
                    try:
                        move = int(input("\nYour turn! Choose a row (1-8): ")) - 1
                        if not self.is_valid_move(move):
                            print("Invalid move. That row is full or doesn't exist.")
                    except ValueError:
                        print("Please enter a number between 1 and 8.")

                self.place_disc(move, 2)
                self.display_board()
            else:
                # AI's turn
                print("\nAI is thinking...")
                sleep(0.5)  # Reduced delay for better UX
                move = self.get_ai_move(1)

                if self.is_valid_move(move):
                    self.place_disc(move, 1)
                    print(f"AI places a disc in row {move + 1}")
                    self.display_board()
                else:
                    print("AI couldn't find a valid move!")

            if self.game_over:
                break

            human_turn = not human_turn  # Switch turns

        # Game over message
        if self.winner == 1:
            print("\nThe AI wins!")
        elif self.winner == 2:
            print("\nCongratulations! You win!")
        else:
            print("\nIt's a draw!")

    def play_ai_vs_ai(self):
        """Play a game of Connect Six: AI vs AI"""
        print("\nAI vs AI demonstration")
        self.display_board()

        current_player = 1  # Player 1 starts

        while not self.game_over:
            print(f"\nAI Player {current_player} is thinking...")
            sleep(0.5)

            move = self.get_ai_move(current_player)

            if self.is_valid_move(move):
                self.place_disc(move, current_player)
                print(f"AI Player {current_player} places in row {move + 1}")
                self.display_board()
            else:
                print(f"AI Player {current_player} couldn't find a valid move!")
                break

            if self.game_over:
                break

            current_player = 3 - current_player  # Switch between 1 and 2

        if self.winner == 1:
            print("\nAI Player 1 (Red) wins!")
        elif self.winner == 2:
            print("\nAI Player 2 (Yellow) wins!")
        else:
            print("\nIt's a draw!")

def display_title():
    """Display the game title banner"""
    print("""

     /$$$$$$                                                      /$$          /$$$$$$  /$$
    /$$__  $$                                                    | $$         /$$__  $$|__/
   | $$  \__/  /$$$$$$  /$$$$$$$  /$$$$$$$   /$$$$$$   /$$$$$$$ /$$$$$$     | $$  \__/ /$$ _____    ____
   | $$       /$$__  $$| $$__  $$| $$__  $$ /$$__  $$ /$$_____/|_  $$_/     |  $$$$$$ | $$|\ $$$$\/$$$$$
   | $$      | $$  \ $$| $$  \ $$| $$  \ $$| $$$$$$$$| $$        | $$        \____  $$| $$| \_$$$  $$$$
   | $$    $$| $$  | $$| $$  | $$| $$  | $$| $$_____/| $$        | $$ /$$    /$$  \ $$| $$|____/$   $_____
   |  $$$$$$/|  $$$$$$/| $$  | $$| $$  | $$|  $$$$$$$|  $$$$$$$  |  $$$$/   |  $$$$$$/| $$| $$$$ /\  $$$$/
    \______/  \______/ |__/  |__/|__/  |__/ \_______/ \_______/   \___/      \______/ |__/ \____/  \____/

    """)
    print("\nWelcome to Connect Six!")
    print("Place discs on an 8x9 board and try to get 6 in a row to win!")

def main():
    """Main function to run the Connect Six game"""
    display_title()

    while True:
        print("\nGame Modes:")
        print("1. Human vs AI")
        print("2. AI vs AI demonstration")
        print("3. Human vs Human")
        print("4. Exit")

        choice = input("\nSelect a game mode (1-4): ").strip()

        if choice == '1':
            game = ConnectSixGame()
            game.play_human_vs_ai()
        elif choice == '2':
            game = ConnectSixGame()
            game.play_ai_vs_ai()
        elif choice == '3':
            game = ConnectSixGame()
            game.play_human_vs_human()
        elif choice == '4':
            print("\nThank you for playing Connect Six! Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

        play_again = input("\nPlay again? (y/n): ").lower()
        if play_again != 'y':
            print("\nThank you for playing Connect Six! Goodbye!")
            break

if __name__ == "__main__":
    main()